Importing data

In [1]:
# uncomment if using this code in RStudio
# setwd(dirname(rstudioapi::getActiveDocumentContext()$path))
# install.packages('coin')

library(coin)
library(lme4)
library(lmerTest)
library(plyr)
library(xtable)
library(LMERConvenienceFunctions)
library(car)
# library(lmerTest) see also https://link.springer.com/article/10.3758/s13428-016-0809-y 

source("r_utils/mer-utils.R")
source("r_utils/regression-utils.R")

data_path = "C:/Users/Arkady/Google Drive/data/eyehandrdk/%s/processed/choices.txt"
output_table_path = "C:/Users/Arkady/Dropbox/Research/Journal papers/2018 CoM/tables/"

choice.data_1 = read.table(sprintf(data_path, "exp_1"), header=T)
choice.data_2 = read.table(sprintf(data_path, "exp_2"), header=T)

Warning message:
"package 'coin' was built under R version 3.5.2"Loading required package: survival
Loading required package: Matrix
Warning message:
"package 'lmerTest' was built under R version 3.5.2"
Attaching package: 'lmerTest'

The following object is masked from 'package:lme4':

    lmer

The following object is masked from 'package:stats':

    step

Loading required package: carData


A bit of preprocessing to create variables for analyzing sequential effects

In [2]:
preprocess_data <- function(choice.data) {
    # any exclusions?
    stats.df = choice.data
    stats.df$subj_id = as.factor(stats.df$subj_id)

    # generate trial number
    max.trial = max(stats.df$trial_no)
    max.block = max(stats.df$block_n)
    stats.df$all_trial_no = (stats.df$session_no-1) * (max.block * max.trial) +
      (stats.df$block_no-1) * max.trial +
      stats.df$trial_no

    ## checking - can be commmented when not needed
    check.diffs = diff(stats.df$all_trial_no)
#     hist(check.diffs)
    check.diffs[check.diffs<0] # 3 resets
    check.diffs[check.diffs>1] # 36 diffs of +2 

    check = stats.df[c(check.diffs>1, T),] # check missing trials
    aggregate(all_trial_no~subj_id, check, length) # - poss cleaning?

    rm(check) # no need 
    ## checking complete

    # generate trial_unique
    stats.df$trial_unique = paste(stats.df$subj_id, stats.df$all_trial_no, sep = "_")

    # generate lasttrial.resp and lasttrial.dir
    lasttrial.details  = ddply(stats.df,
                               .(subj_id, session_no, block_no), summarize,
                               trial_no = trial_no,
                               response = response,
                               lasttrial.resp  = c(F, diff(response)==0),
                               direction = direction,
                               lasttrial.dir = c(F, diff(direction)==0),
                               trial_unique = trial_unique
    )

    # add lasttrial.resp and lasttrial.dir to stats.df
    stats.df = merge(stats.df, lasttrial.details[,
                        c("trial_unique", "lasttrial.resp", "lasttrial.dir")],
                        by = "trial_unique")

    # set up contrasts for lasttrial.resp and lasttrial.dir 
    contrasts(stats.df$lasttrial.resp) = contr.sum(n=2) /2 *-1
    contrasts(stats.df$lasttrial.resp) # true is pos

    contrasts(stats.df$lasttrial.dir) = contr.sum(n=2) /2 *-1
    contrasts(stats.df$lasttrial.dir) # true is pos

    contrasts(stats.df$is_com) = contr.sum(n=2) /2 *-1
    contrasts(stats.df$is_com)# true is pos
    
    stats.df = stats.df[(stats.df$hand_IT < 1.5) & (stats.df$eye_IT < 1.5),]

    str(stats.df)

    return(stats.df)
}

stats.df_1 = preprocess_data(choice.data_1)
stats.df_2 = preprocess_data(choice.data_2)

'data.frame':	9157 obs. of  50 variables:
 $ trial_unique              : chr  "269_100" "269_1000" "269_1001" "269_1002" ...
 $ subj_id                   : Factor w/ 4 levels "269","275","391",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ session_no                : int  1 2 2 2 2 2 2 2 2 2 ...
 $ block_no                  : int  2 7 7 7 7 7 7 7 7 7 ...
 $ trial_no                  : int  40 40 41 42 43 44 45 46 47 48 ...
 $ is_practice               : Factor w/ 2 levels "False","True": 2 2 2 2 2 2 2 2 2 2 ...
 $ direction                 : num  180 180 0 180 0 180 0 0 0 0 ...
 $ coherence                 : num  0.064 0 0.032 0.064 0.256 0.256 0.032 0.032 0.128 0.064 ...
 $ duration                  : int  800 800 800 800 800 800 800 800 800 800 ...
 $ response                  : int  0 180 0 180 0 180 0 0 0 180 ...
 $ response_time             : num  1.147 0.935 1.208 1.318 0.785 ...
 $ is_correct                : Factor w/ 2 levels "False","True": 1 2 2 2 2 2 2 2 2 1 ...
 $ xflips                  

# Analysis 1. Accuracy as a function of coherence

In [3]:
rnd_effects_analysis_1 <- function(stats.df){
    rnd.lmer = glmer(is_correct ~ (1|subj_id), 
                 stats.df[stats.df$coherence!=0,], # edit this line to include zero coh
                 family = binomial)

    # rnd intercept for each participant and random slope of trials
    # diff avg acc, diff learning effect for each p
    rnd2.lmer = glmer(is_correct ~ (c.(l.(all_trial_no))|subj_id), 
                      stats.df[stats.df$coherence!=0,], 
                      family = binomial)

    # rnd intercept for each participant and random slope of coherence
    # diff avg acc, diff coherence effect for each p
    rnd3.lmer = glmer(is_correct ~ (c.(coherence)|subj_id), 
                      stats.df[stats.df$coherence!=0,], 
                      family = binomial)

    # rnd intercept for each participant and random slope of trials
    # diff avg acc, diff linear and quad learning effect for each p
    rnd3.quad.lmer = glmer(is_correct ~ (poly(coherence, 2, raw = T)|subj_id), 
                           stats.df[stats.df$coherence!=0,], 
                           family = binomial)

    # rnd intercept for each participant and random slope of trials
    # diff avg acc, diff trial-to-trial stimulus effect for each p
    rnd4.lmer = glmer(is_correct ~ (lasttrial.dir|subj_id), 
                      stats.df[stats.df$coherence!=0,], 
                      family = binomial)

    # rnd intercept for each participant and random slope of trials
    # diff avg acc, diff trial-to-trial stimulus effect and diff learning effect for each p
    rnd5.lmer = glmer(is_correct ~ (c.(l.(all_trial_no)) + c.(coherence)|subj_id), 
                      stats.df[stats.df$coherence!=0,], 
                      family = binomial)

    # rnd.lmer with com and coherence
    rnd6.lmer = glmer(is_correct ~ ((is_com + c.(coherence))|subj_id),
                      stats.df[stats.df$coherence!=0,],
                      family = binomial)

    rnd.anova = anova(rnd.lmer, rnd2.lmer, rnd3.lmer, rnd3.quad.lmer, rnd4.lmer, rnd5.lmer, rnd6.lmer)
    print(rnd.anova)
    
    print("Best model according to AIC")
    print(row.names(rnd.anova[rnd.anova$AIC==min(rnd.anova$AIC), ]))
    print("Best model according to BIC")
    print(row.names(rnd.anova[rnd.anova$BIC==min(rnd.anova$BIC), ]))
}

In [4]:
rnd_effects_analysis_1(stats.df_1)

singular fit
singular fit
singular fit
singular fit


Data: stats.df[stats.df$coherence != 0, ]
Models:
rnd.lmer: is_correct ~ (1 | subj_id)
rnd2.lmer: is_correct ~ (c.(l.(all_trial_no)) | subj_id)
rnd3.lmer: is_correct ~ (c.(coherence) | subj_id)
rnd4.lmer: is_correct ~ (lasttrial.dir | subj_id)
rnd3.quad.lmer: is_correct ~ (poly(coherence, 2, raw = T) | subj_id)
rnd5.lmer: is_correct ~ (c.(l.(all_trial_no)) + c.(coherence) | subj_id)
rnd6.lmer: is_correct ~ ((is_com + c.(coherence)) | subj_id)
               Df    AIC    BIC  logLik deviance     Chisq Chi Df Pr(>Chisq)
rnd.lmer        2 8577.7 8591.6 -4286.9   8573.7                            
rnd2.lmer       4 8580.0 8607.8 -4286.0   8572.0    1.7031      2     0.4267
rnd3.lmer       4 7186.5 7214.3 -3589.3   7178.5 1393.4849      0     <2e-16
rnd4.lmer       4 8459.2 8486.9 -4225.6   8451.2    0.0000      0     1.0000
rnd3.quad.lmer  7 7189.9 7238.5 -3587.9   7175.9 1275.2921      3     <2e-16
rnd5.lmer       7 7205.9 7254.5 -3596.0   7191.9    0.0000      0     1.0000
rnd6.lmer     

In [5]:
rnd_effects_analysis_1(stats.df_2)

singular fit
singular fit
singular fit
singular fit


Data: stats.df[stats.df$coherence != 0, ]
Models:
rnd.lmer: is_correct ~ (1 | subj_id)
rnd2.lmer: is_correct ~ (c.(l.(all_trial_no)) | subj_id)
rnd3.lmer: is_correct ~ (c.(coherence) | subj_id)
rnd4.lmer: is_correct ~ (lasttrial.dir | subj_id)
rnd3.quad.lmer: is_correct ~ (poly(coherence, 2, raw = T) | subj_id)
rnd5.lmer: is_correct ~ (c.(l.(all_trial_no)) + c.(coherence) | subj_id)
rnd6.lmer: is_correct ~ ((is_com + c.(coherence)) | subj_id)
               Df    AIC    BIC  logLik deviance    Chisq Chi Df Pr(>Chisq)    
rnd.lmer        2 6473.3 6486.6 -3234.6   6469.3                               
rnd2.lmer       4 6458.8 6485.5 -3225.4   6450.8  18.4541      2  9.834e-05 ***
rnd3.lmer       4 5574.3 5600.9 -2783.1   5566.3 884.5557      0  < 2.2e-16 ***
rnd4.lmer       4 6462.1 6488.8 -3227.1   6454.1   0.0000      0          1    
rnd3.quad.lmer  7 5577.9 5624.5 -2781.9   5563.9 890.2484      3  < 2.2e-16 ***
rnd5.lmer       7 5556.1 5602.7 -2771.0   5542.1  21.7666      0  < 2.2e-

In both experiments, rnd3 and rnd6 are among the best, so we might want to include is_com and coherence as random slopes, but unfortunately both models don't converge when full effects are included. So we include only random intercept

In [37]:
run_analysis_1 <- function(stats.df, exp_name){
    choice.mer = glmer(is_correct ~ (1 | subj_id) + c.(coherence)*is_com + c.(l.(all_trial_no)) + lasttrial.dir + lasttrial.resp,
                      stats.df[stats.df$coherence!=0,],
                      family = binomial)
    print(summary(choice.mer))

    choice.output = summary(choice.mer)$coefficients

    file_name = sprintf(paste(output_table_path, "is_correct_vs_coh_%s.tex", sep=""), exp_name)
    print(xtable(choice.output, digits = c(2,2,2,2,-2), 
                 label = "tab:is_correct_vs_coh",
                 caption = 'Accuracy as a function of coherence'), 
          math.style.exponents = TRUE, type = "latex", file = file_name)
}

In [38]:
run_analysis_1(stats.df_1, "exp_1")

Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: 
is_correct ~ (1 | subj_id) + c.(coherence) * is_com + c.(l.(all_trial_no)) +  
    lasttrial.dir + lasttrial.resp
   Data: stats.df[stats.df$coherence != 0, ]

     AIC      BIC   logLik deviance df.resid 
  7092.3   7147.9  -3538.2   7076.3     7665 

Scaled residuals: 
     Min       1Q   Median       3Q      Max 
-12.5023   0.0794   0.3055   0.6431   1.4228 

Random effects:
 Groups  Name        Variance Std.Dev.
 subj_id (Intercept) 0.04706  0.2169  
Number of obs: 7673, groups:  subj_id, 4

Fixed effects:
                      Estimate Std. Error z value Pr(>|z|)    
(Intercept)            1.24884    0.17403   7.176 7.18e-13 ***
c.(coherence)          7.70135    1.05738   7.283 3.25e-13 ***
is_com1               -0.82917    0.27275  -3.040  0.00237 ** 
c.(l.(all_trial_no))   0.06925    0.02933   2.361  0.01823 *  
lasttrial.dir1         0.57547    0.

In [39]:
run_analysis_1(stats.df_2, "exp_2")

Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: 
is_correct ~ (1 | subj_id) + c.(coherence) * is_com + c.(l.(all_trial_no)) +  
    lasttrial.dir + lasttrial.resp
   Data: stats.df[stats.df$coherence != 0, ]

     AIC      BIC   logLik deviance df.resid 
  5498.9   5552.2  -2741.5   5482.9     5784 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-9.3177  0.1075  0.3422  0.6590  1.6964 

Random effects:
 Groups  Name        Variance Std.Dev.
 subj_id (Intercept) 0.001421 0.03769 
Number of obs: 5792, groups:  subj_id, 3

Fixed effects:
                      Estimate Std. Error z value Pr(>|z|)    
(Intercept)            0.89147    0.08994   9.912  < 2e-16 ***
c.(coherence)          4.68684    0.56441   8.304  < 2e-16 ***
is_com1               -1.41333    0.17386  -8.129 4.33e-16 ***
c.(l.(all_trial_no))   0.16078    0.03326   4.834 1.34e-06 ***
lasttrial.dir1         0.32800    0.06703   4.

# Analysis 2. Probability of CoM as a function of initiation time

In [9]:
rnd_effects_analysis_2 <- function(stats.df){
    stats.df <- stats.df[complete.cases(stats.df[ , "eye_IT"]),]
    rnd1.lmer = glmer(is_com ~ (1|subj_id), stats.df, family = binomial)

    rnd2.lmer = glmer(is_com ~ (c.(coherence)|subj_id), stats.df, family = binomial)

    rnd3.lmer = glmer(is_com ~ (poly(c.(coherence), 2, raw = T)|subj_id), stats.df, family = binomial)

    rnd4.lmer = glmer(is_com ~ (c.(hand_IT)|subj_id), stats.df, family = binomial)
    
    rnd5.lmer = glmer(is_com ~ (c.(eye_IT)|subj_id), stats.df, family = binomial)
    
    rnd6.lmer = glmer(is_com ~ (c.(hand_IT)*c.(coherence)|subj_id), stats.df, family = binomial)
    
    rnd7.lmer = glmer(is_com ~ (c.(eye_IT)*c.(coherence)|subj_id), stats.df, family = binomial)
    
    rnd8.lmer = glmer(is_com ~ (c.(eye_IT)*c.(coherence) + c.(hand_IT)*c.(coherence)|subj_id), stats.df, family = binomial)
    
    rnd.anova = anova(rnd1.lmer, rnd2.lmer, rnd3.lmer, rnd4.lmer, rnd5.lmer, rnd6.lmer, rnd7.lmer, rnd8.lmer)
#     rnd.anova = anova(rnd4.lmer, rnd5.lmer)
    print(rnd.anova)
    
    print("Best model according to AIC")
    print(row.names(rnd.anova[rnd.anova$AIC==min(rnd.anova$AIC), ]))
    print("Best model according to BIC")
    print(row.names(rnd.anova[rnd.anova$BIC==min(rnd.anova$BIC), ]))
}

In [10]:
rnd_effects_analysis_2(stats.df_1)

singular fit
Warning message in checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv, :
"Model failed to converge with max|grad| = 0.00201429 (tol = 0.001, component 1)"Warning message in checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv, :
"Model failed to converge with max|grad| = 0.00128922 (tol = 0.001, component 1)"

Data: stats.df
Models:
rnd1.lmer: is_com ~ (1 | subj_id)
rnd2.lmer: is_com ~ (c.(coherence) | subj_id)
rnd4.lmer: is_com ~ (c.(hand_IT) | subj_id)
rnd5.lmer: is_com ~ (c.(eye_IT) | subj_id)
rnd3.lmer: is_com ~ (poly(c.(coherence), 2, raw = T) | subj_id)
rnd6.lmer: is_com ~ (c.(hand_IT) * c.(coherence) | subj_id)
rnd7.lmer: is_com ~ (c.(eye_IT) * c.(coherence) | subj_id)
rnd8.lmer: is_com ~ (c.(eye_IT) * c.(coherence) + c.(hand_IT) * c.(coherence) | 
rnd8.lmer:     subj_id)
          Df    AIC    BIC  logLik deviance  Chisq Chi Df Pr(>Chisq)    
rnd1.lmer  2 1983.6 1997.8 -989.79   1979.6                             
rnd2.lmer  4 1929.6 1958.0 -960.78   1921.6 58.016      2  2.523e-13 ***
rnd4.lmer  4 1853.2 1881.7 -922.61   1845.2 76.352      0  < 2.2e-16 ***
rnd5.lmer  4 1900.7 1929.2 -946.36   1892.7  0.000      0          1    
rnd3.lmer  7 1916.3 1966.1 -951.14   1902.3  0.000      3          1    
rnd6.lmer 11 1847.6 1925.9 -912.78   1825.6 76.714      4  8.646e-16 ***
rnd7.lmer 1

In [11]:
rnd_effects_analysis_2(stats.df_2)

singular fit
singular fit
Warning message in optwrap(optimizer, devfun, start, rho$lower, control = control, :
"convergence code 1 from bobyqa: bobyqa -- maximum number of function evaluations exceeded"

Data: stats.df
Models:
rnd1.lmer: is_com ~ (1 | subj_id)
rnd2.lmer: is_com ~ (c.(coherence) | subj_id)
rnd4.lmer: is_com ~ (c.(hand_IT) | subj_id)
rnd5.lmer: is_com ~ (c.(eye_IT) | subj_id)
rnd3.lmer: is_com ~ (poly(c.(coherence), 2, raw = T) | subj_id)
rnd6.lmer: is_com ~ (c.(hand_IT) * c.(coherence) | subj_id)
rnd7.lmer: is_com ~ (c.(eye_IT) * c.(coherence) | subj_id)
rnd8.lmer: is_com ~ (c.(eye_IT) * c.(coherence) + c.(hand_IT) * c.(coherence) | 
rnd8.lmer:     subj_id)
          Df    AIC    BIC  logLik deviance   Chisq Chi Df Pr(>Chisq)    
rnd1.lmer  2 1900.7 1914.4 -948.35   1896.7                              
rnd2.lmer  4 1884.2 1911.6 -938.12   1876.2  20.454      2  3.619e-05 ***
rnd4.lmer  4 1889.3 1916.7 -940.66   1881.3   0.000      0  1.0000000    
rnd5.lmer  4 1778.2 1805.6 -885.09   1770.2 111.136      0  < 2.2e-16 ***
rnd3.lmer  7 1887.9 1935.9 -936.96   1873.9   0.000      3  1.0000000    
rnd6.lmer 11 1879.9 1955.3 -928.97   1857.9  15.986      4  0.0030374 ** 
rnd7

Assuming that we have to pick up a universal random effect structure which works for both experiments, we need to choose between a coherence-only random slope (rnd2), or maximal model (rnd8), which fares better among two in terms of AIC and BIC. However, both models doen't converge, so we're left with random intercept

In [42]:
run_analysis_2 <- function(stats.df, exp_name){
    choice.mer = glmer(is_com ~ (1 | subj_id) + c.(coherence)*c.(hand_IT) + c.(coherence)*c.(eye_IT),
                      stats.df,
                      family = binomial)
    print(summary(choice.mer))

    # confidence intervals if required
    CIs = confint(choice.mer, parm="beta_",method="Wald")
    CIs.tab = cbind(est.OR=fixef(choice.mer),CIs)
    Odds.tab = exp(CIs.tab)
    print(Odds.tab,digits=3)

    choice.output = summary(choice.mer)$coefficients

    file_name = sprintf(paste(output_table_path, "is_com_vs_IT_%s.tex", sep=""), exp_name)
    print(xtable(choice.output, digits = c(2,2,2,2,-2), 
                 label = "tab:is_com_vs_IT",
                 caption = 'Probability of change-of-mind as a function of hand and eye initiation times'), 
          math.style.exponents = TRUE, type = "latex", file = file_name)
}

In [43]:
run_analysis_2(stats.df_1, "exp_1")

Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: 
is_com ~ (1 | subj_id) + c.(coherence) * c.(hand_IT) + c.(coherence) *  
    c.(eye_IT)
   Data: stats.df

     AIC      BIC   logLik deviance df.resid 
  1851.1   1900.9   -918.5   1837.1     9146 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-0.8662 -0.1944 -0.0967 -0.0558 26.6968 

Random effects:
 Groups  Name        Variance Std.Dev.
 subj_id (Intercept) 1.525    1.235   
Number of obs: 9153, groups:  subj_id, 4

Fixed effects:
                          Estimate Std. Error z value Pr(>|z|)    
(Intercept)                -4.4444     0.6311  -7.042 1.89e-12 ***
c.(coherence)              -2.5286     0.6482  -3.901 9.58e-05 ***
c.(hand_IT)                 1.1660     0.3224   3.616 0.000299 ***
c.(eye_IT)                  1.3101     0.4536   2.888 0.003872 ** 
c.(coherence):c.(hand_IT)   2.7056     2.3020   1.175 0.239855    
c.(coherence

In [44]:
run_analysis_2(stats.df_2, "exp_2")

Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: 
is_com ~ (1 | subj_id) + c.(coherence) * c.(hand_IT) + c.(coherence) *  
    c.(eye_IT)
   Data: stats.df

     AIC      BIC   logLik deviance df.resid 
  1751.7   1799.6   -868.9   1737.7     6952 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-1.2418 -0.1896 -0.1515 -0.1153 14.8988 

Random effects:
 Groups  Name        Variance Std.Dev.
 subj_id (Intercept) 0.2347   0.4845  
Number of obs: 6959, groups:  subj_id, 3

Fixed effects:
                          Estimate Std. Error z value Pr(>|z|)    
(Intercept)                -3.7442     0.2936 -12.752  < 2e-16 ***
c.(coherence)              -1.8016     0.5725  -3.147  0.00165 ** 
c.(hand_IT)                -1.0290     0.5349  -1.924  0.05440 .  
c.(eye_IT)                  5.3951     0.4800  11.239  < 2e-16 ***
c.(coherence):c.(hand_IT)   4.8544     2.8990   1.675  0.09403 .  
c.(coherence

# Analysis 3. Initiation times as a function of coherence

## Hand IT: random effects structure

In [15]:
rnd_effects_analysis_3_hand <- function(stats.df){
#     stats.df <- stats.df[complete.cases(stats.df[ , "eye_IT"]),]
    rnd1.lmer = lmer(hand_IT ~ (1|subj_id), stats.df)

    rnd2.lmer = lmer(hand_IT ~ (c.(coherence)|subj_id), stats.df)

    rnd3.lmer = lmer(hand_IT ~ (is_correct|subj_id), stats.df)

    rnd4.lmer = lmer(hand_IT ~ ((c.(coherence)+is_correct)|subj_id), stats.df)
    
    rnd5.lmer = lmer(hand_IT ~ ((c.(coherence)*is_correct)|subj_id), stats.df)
    
    rnd.anova = anova(rnd1.lmer, rnd2.lmer, rnd3.lmer, rnd4.lmer, rnd5.lmer)
#     rnd.anova = anova(rnd4.lmer, rnd5.lmer)
    print(rnd.anova)
    print("Best model according to AIC")
    print(row.names(rnd.anova[rnd.anova$AIC==min(rnd.anova$AIC), ]))
    print("Best model according to BIC")
    print(row.names(rnd.anova[rnd.anova$BIC==min(rnd.anova$BIC), ]))
}

In [16]:
rnd_effects_analysis_3_hand(stats.df_1)

singular fit
singular fit
singular fit
refitting model(s) with ML (instead of REML)


Data: stats.df
Models:
rnd1.lmer: hand_IT ~ (1 | subj_id)
rnd2.lmer: hand_IT ~ (c.(coherence) | subj_id)
rnd3.lmer: hand_IT ~ (is_correct | subj_id)
rnd4.lmer: hand_IT ~ ((c.(coherence) + is_correct) | subj_id)
rnd5.lmer: hand_IT ~ ((c.(coherence) * is_correct) | subj_id)
          Df    AIC    BIC  logLik deviance   Chisq Chi Df Pr(>Chisq)    
rnd1.lmer  3 4830.2 4851.6 -2412.1   4824.2                              
rnd2.lmer  5 2546.3 2581.9 -1268.2   2536.3 2287.90      2     <2e-16 ***
rnd3.lmer  5 4162.2 4197.8 -2076.1   4152.2    0.00      0          1    
rnd4.lmer  8 2442.5 2499.5 -1213.2   2426.5 1725.68      3     <2e-16 ***
rnd5.lmer 12 2330.3 2415.7 -1153.1   2306.3  120.23      4     <2e-16 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1
[1] "Best model according to AIC"
[1] "rnd5.lmer"
[1] "Best model according to BIC"
[1] "rnd5.lmer"


In [17]:
rnd_effects_analysis_3_hand(stats.df_2)

singular fit
singular fit
refitting model(s) with ML (instead of REML)


Data: stats.df
Models:
rnd1.lmer: hand_IT ~ (1 | subj_id)
rnd2.lmer: hand_IT ~ (c.(coherence) | subj_id)
rnd3.lmer: hand_IT ~ (is_correct | subj_id)
rnd4.lmer: hand_IT ~ ((c.(coherence) + is_correct) | subj_id)
rnd5.lmer: hand_IT ~ ((c.(coherence) * is_correct) | subj_id)
          Df     AIC     BIC logLik deviance    Chisq Chi Df Pr(>Chisq)    
rnd1.lmer  3 -5370.0 -5349.5 2688.0  -5376.0                               
rnd2.lmer  5 -6370.2 -6336.0 3190.1  -6380.2 1004.166      2  < 2.2e-16 ***
rnd3.lmer  5 -5708.0 -5673.8 2859.0  -5718.0    0.000      0          1    
rnd4.lmer  8 -6447.0 -6392.2 3231.5  -6463.0  745.005      3  < 2.2e-16 ***
rnd5.lmer 12 -6503.9 -6421.7 3263.9  -6527.9   64.867      4  2.745e-13 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1
[1] "Best model according to AIC"
[1] "rnd5.lmer"
[1] "Best model according to BIC"
[1] "rnd5.lmer"


For both experiments, rnd5 is the best, but it doesn't converge for Exp 2, so we use rnd4

## Hand IT: analysis

In [48]:
run_analysis_3_hand <- function(stats.df, exp_name){
    choice.mer = lmer(hand_IT ~ ((c.(coherence)+is_correct) | subj_id) + c.(coherence)*is_correct, stats.df)
    print(summary(choice.mer))

    choice.output = summary(choice.mer)$coefficients

    file_name = sprintf(paste(output_table_path, "hand_IT_vs_coh_%s.tex", sep=""), exp_name)
    print(xtable(choice.output, digits = c(2,2,2,2,2,-2), 
                 label = "tab:hand_IT_vs_coh",
                 caption = 'Hand initiation time as a function of stimulus strength'), 
          math.style.exponents = TRUE, type = "latex", file = file_name)
}

In [49]:
run_analysis_3_hand(stats.df_1, "exp_1")

singular fit


Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: hand_IT ~ ((c.(coherence) + is_correct) | subj_id) + c.(coherence) *  
    is_correct
   Data: stats.df

REML criterion at convergence: 2321.6

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-3.5509 -0.5970 -0.1294  0.3916  4.7868 

Random effects:
 Groups   Name           Variance Std.Dev. Corr       
 subj_id  (Intercept)    0.007164 0.08464             
          c.(coherence)  0.242607 0.49255  -0.94      
          is_correctTrue 0.001007 0.03174  -0.91  0.71
 Residual                0.075070 0.27399             
Number of obs: 9153, groups:  subj_id, 4

Fixed effects:
                               Estimate Std. Error         df t value Pr(>|t|)
(Intercept)                     0.43200    0.04326    3.14828   9.987  0.00173
c.(coherence)                   0.05368    0.25541    3.43394   0.210  0.84540
is_correctTrue                 -0.14692    0.01855    4.05590  -7.921  0.0

In [50]:
run_analysis_3_hand(stats.df_2, "exp_2")

singular fit


Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: hand_IT ~ ((c.(coherence) + is_correct) | subj_id) + c.(coherence) *  
    is_correct
   Data: stats.df

REML criterion at convergence: -6488.9

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-4.4086 -0.6173 -0.0694  0.4699  7.3069 

Random effects:
 Groups   Name           Variance  Std.Dev. Corr     
 subj_id  (Intercept)    0.0110503 0.10512           
          c.(coherence)  0.0363761 0.19073  0.58     
          is_correctTrue 0.0002704 0.01645  0.03 0.84
 Residual                0.0228775 0.15125           
Number of obs: 6959, groups:  subj_id, 3

Fixed effects:
                               Estimate Std. Error         df t value Pr(>|t|)
(Intercept)                     0.32436    0.06090    2.01273   5.326   0.0330
c.(coherence)                  -0.03936    0.11660    2.46320  -0.338   0.7623
is_correctTrue                 -0.05958    0.01098    2.56749  -5.427   0.0182

## Eye IT: Random effects structure

In [21]:
rnd_effects_analysis_3_eye <- function(stats.df){
    rnd1.lmer = lmer(eye_IT ~ (1|subj_id), stats.df)

    rnd2.lmer = lmer(eye_IT ~ (c.(coherence)|subj_id), stats.df)

    rnd3.lmer = lmer(eye_IT ~ (is_correct|subj_id), stats.df)

    rnd4.lmer = lmer(eye_IT ~ ((c.(coherence) + is_correct)|subj_id), stats.df)
    
    rnd5.lmer = lmer(eye_IT ~ ((c.(coherence)*is_correct)|subj_id), stats.df)
    
    rnd.anova = anova(rnd1.lmer, rnd2.lmer, rnd3.lmer, rnd4.lmer, rnd5.lmer)
    print(rnd.anova)
    print("Best model according to AIC")
    print(row.names(rnd.anova[rnd.anova$AIC==min(rnd.anova$AIC), ]))
    print("Best model according to BIC")
    print(row.names(rnd.anova[rnd.anova$BIC==min(rnd.anova$BIC), ]))
}

In [22]:
rnd_effects_analysis_3_eye(stats.df_1)

singular fit
refitting model(s) with ML (instead of REML)


Data: stats.df
Models:
rnd1.lmer: eye_IT ~ (1 | subj_id)
rnd2.lmer: eye_IT ~ (c.(coherence) | subj_id)
rnd3.lmer: eye_IT ~ (is_correct | subj_id)
rnd4.lmer: eye_IT ~ ((c.(coherence) + is_correct) | subj_id)
rnd5.lmer: eye_IT ~ ((c.(coherence) * is_correct) | subj_id)
          Df     AIC     BIC logLik deviance   Chisq Chi Df Pr(>Chisq)    
rnd1.lmer  3 -2096.1 -2074.8 1051.1  -2102.1                              
rnd2.lmer  5 -3164.4 -3128.7 1587.2  -3174.4 1072.21      2     <2e-16 ***
rnd3.lmer  5 -2487.0 -2451.4 1248.5  -2497.0    0.00      0          1    
rnd4.lmer  8 -3241.5 -3184.6 1628.8  -3257.5  760.57      3     <2e-16 ***
rnd5.lmer 12 -3348.9 -3263.4 1686.5  -3372.9  115.37      4     <2e-16 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1
[1] "Best model according to AIC"
[1] "rnd5.lmer"
[1] "Best model according to BIC"
[1] "rnd5.lmer"


In [23]:
rnd_effects_analysis_3_eye(stats.df_2)

singular fit
singular fit
refitting model(s) with ML (instead of REML)


Data: stats.df
Models:
rnd1.lmer: eye_IT ~ (1 | subj_id)
rnd2.lmer: eye_IT ~ (c.(coherence) | subj_id)
rnd3.lmer: eye_IT ~ (is_correct | subj_id)
rnd4.lmer: eye_IT ~ ((c.(coherence) + is_correct) | subj_id)
rnd5.lmer: eye_IT ~ ((c.(coherence) * is_correct) | subj_id)
          Df     AIC     BIC logLik deviance   Chisq Chi Df Pr(>Chisq)    
rnd1.lmer  3 -7595.3 -7574.8 3800.7  -7601.3                              
rnd2.lmer  5 -8424.6 -8390.3 4217.3  -8434.6 833.230      2     <2e-16 ***
rnd3.lmer  5 -7978.4 -7944.2 3994.2  -7988.4   0.000      0          1    
rnd4.lmer  8 -8553.8 -8499.0 4284.9  -8569.8 581.394      3     <2e-16 ***
rnd5.lmer 12 -8637.3 -8555.2 4330.7  -8661.3  91.545      4     <2e-16 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1
[1] "Best model according to AIC"
[1] "rnd5.lmer"
[1] "Best model according to BIC"
[1] "rnd5.lmer"


In both experiments, rnd5 works great, and the resulting model actually converges

## Eye IT: Analysis

In [51]:
run_analysis_3_eye <- function(stats.df, exp_name){
    choice.mer = lmer(eye_IT ~ ((c.(coherence)*is_correct)|subj_id) + c.(coherence)*is_correct, stats.df)
    print(summary(choice.mer))

    choice.output = summary(choice.mer)$coefficients

    file_name = sprintf(paste(output_table_path, "eye_IT_vs_coh_%s.tex", sep=""), exp_name)
    print(xtable(choice.output, digits = c(2,2,2,2,2,-2), 
                 label = "tab:eye_IT_vs_coh",
                 caption = 'Eye initiation time as a function of stimulus strength'), 
          math.style.exponents = TRUE, type = "latex", file = file_name)
}

In [52]:
run_analysis_3_eye(stats.df_1, "exp_1")

singular fit


Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: eye_IT ~ ((c.(coherence) * is_correct) | subj_id) + c.(coherence) *  
    is_correct
   Data: stats.df

REML criterion at convergence: -3362.2

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-2.4366 -0.5844 -0.2234  0.2850  5.9121 

Random effects:
 Groups   Name                         Variance  Std.Dev. Corr             
 subj_id  (Intercept)                  0.0055674 0.07462                   
          c.(coherence)                0.0395148 0.19878   0.44            
          is_correctTrue               0.0008091 0.02845  -0.85 -0.16      
          c.(coherence):is_correctTrue 0.0306329 0.17502  -0.94 -0.69  0.79
 Residual                              0.0403073 0.20077                   
Number of obs: 9153, groups:  subj_id, 4

Fixed effects:
                             Estimate Std. Error       df t value Pr(>|t|)   
(Intercept)                   0.39369    0.03792  2.

In [53]:
run_analysis_3_eye(stats.df_2, "exp_2")

singular fit


Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: eye_IT ~ ((c.(coherence) * is_correct) | subj_id) + c.(coherence) *  
    is_correct
   Data: stats.df

REML criterion at convergence: -8649.1

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-3.2262 -0.5992 -0.1646  0.2849  6.9653 

Random effects:
 Groups   Name                         Variance  Std.Dev. Corr             
 subj_id  (Intercept)                  0.0080339 0.08963                   
          c.(coherence)                0.0044709 0.06687  -0.87            
          is_correctTrue               0.0007126 0.02670  -0.97  0.97      
          c.(coherence):is_correctTrue 0.0088884 0.09428  -0.77  0.99  0.91
 Residual                              0.0167746 0.12952                   
Number of obs: 6959, groups:  subj_id, 3

Fixed effects:
                             Estimate Std. Error       df t value Pr(>|t|)  
(Intercept)                   0.36109    0.05193  2.0

# Analysis 4. Symmetry of the lags between hand and eye movement

## Lag at initial decision

This is a tricky one, see the plots in the eye_hand_lags notebook. Lag at initial decision is mostly defined by hand initiation time, the baseline for which is highly individual-specific. Overall, the average lag is negative in both experiments:

In [27]:
t.test(stats.df_1$ID_lag)


	One Sample t-test

data:  stats.df_1$ID_lag
t = -5.1241, df = 9152, p-value = 3.051e-07
alternative hypothesis: true mean is not equal to 0
95 percent confidence interval:
 -0.015814980 -0.007062961
sample estimates:
  mean of x 
-0.01143897 


In [28]:
t.test(stats.df_2$ID_lag)


	One Sample t-test

data:  stats.df_2$ID_lag
t = -12.264, df = 6958, p-value < 2.2e-16
alternative hypothesis: true mean is not equal to 0
95 percent confidence interval:
 -0.02768085 -0.02005109
sample estimates:
  mean of x 
-0.02386597 


However, when accounting for individual differences, there is no evidence that population-level intercept is different from zero.

In [29]:
run_analysis_4_ID <- function(stats.df, exp_name){
    choice.mer = lmer(ID_lag ~ (1|subj_id) + 1, stats.df)
    print(summary(choice.mer))

    choice.output = summary(choice.mer)$coefficients
}

In [30]:
run_analysis_4_ID(stats.df_1, "exp_1")

Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: ID_lag ~ (1 | subj_id) + 1
   Data: stats.df

REML criterion at convergence: -3080.5

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-7.3212 -0.4035 -0.0539  0.3350  5.7875 

Random effects:
 Groups   Name        Variance Std.Dev.
 subj_id  (Intercept) 0.005097 0.07139 
 Residual             0.041697 0.20420 
Number of obs: 9153, groups:  subj_id, 4

Fixed effects:
            Estimate Std. Error       df t value Pr(>|t|)
(Intercept) -0.01032    0.03576  3.00061  -0.289    0.792


In [31]:
run_analysis_4_ID(stats.df_2, "exp_2")

Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: ID_lag ~ (1 | subj_id) + 1
   Data: stats.df

REML criterion at convergence: -7896.9

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-5.3924 -0.4577  0.0393  0.4825  7.9482 

Random effects:
 Groups   Name        Variance Std.Dev.
 subj_id  (Intercept) 0.01166  0.108   
 Residual             0.01876  0.137   
Number of obs: 6959, groups:  subj_id, 3

Fixed effects:
            Estimate Std. Error       df t value Pr(>|t|)
(Intercept) -0.02762    0.06238  1.99992  -0.443    0.701


For these reasons, I'd suggest we do not make any general claims regarding the hand-eye lag at initial decision. We can potentially try to account for positive/negative lags in different participants in Nadim's model by fitting it to the individual-level data, but IMO that's not very meaningful

## Lag at change-of-mind decision

Just as the lag at initial decision, the CoM lag is negative overall

In [32]:
t.test(stats.df_1$com_lag)


	One Sample t-test

data:  stats.df_1$com_lag
t = -4.1093, df = 224, p-value = 5.567e-05
alternative hypothesis: true mean is not equal to 0
95 percent confidence interval:
 -0.08374215 -0.02945785
sample estimates:
mean of x 
  -0.0566 


In [33]:
t.test(stats.df_2$com_lag)


	One Sample t-test

data:  stats.df_2$com_lag
t = -5.4062, df = 201, p-value = 1.815e-07
alternative hypothesis: true mean is not equal to 0
95 percent confidence interval:
 -0.06298461 -0.02931836
sample estimates:
  mean of x 
-0.04615149 


In Experiment 1, negativity remains when accounting for individual differences, but, interestingly, not in Experiment 2

In [34]:
run_analysis_4_COM <- function(stats.df, exp_name){
    choice.mer = lmer(com_lag ~ (1|subj_id) + 1, stats.df)
    print(summary(choice.mer))

    choice.output = summary(choice.mer)$coefficients
}

In [35]:
run_analysis_4_COM(stats.df_1, "exp_1")

singular fit


Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: com_lag ~ (1 | subj_id) + 1
   Data: stats.df

REML criterion at convergence: -65.4

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-5.2758 -0.0673  0.2251  0.4201  3.1945 

Random effects:
 Groups   Name        Variance  Std.Dev. 
 subj_id  (Intercept) 7.006e-17 8.370e-09
 Residual             4.268e-02 2.066e-01
Number of obs: 225, groups:  subj_id, 4

Fixed effects:
             Estimate Std. Error        df t value Pr(>|t|)    
(Intercept)  -0.05660    0.01377 224.00000  -4.109 5.57e-05 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1
convergence code: 0
singular fit



In [36]:
run_analysis_4_COM(stats.df_2, "exp_2")

Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: com_lag ~ (1 | subj_id) + 1
   Data: stats.df

REML criterion at convergence: -280.4

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-5.3742 -0.1437  0.2215  0.5430  1.7652 

Random effects:
 Groups   Name        Variance Std.Dev.
 subj_id  (Intercept) 0.001207 0.03474 
 Residual             0.013869 0.11777 
Number of obs: 202, groups:  subj_id, 3

Fixed effects:
            Estimate Std. Error       df t value Pr(>|t|)
(Intercept) -0.04185    0.02174  2.04618  -1.925    0.191
